Result of existing parsing



> SELECT * FROM public.extraction_parse_references
ORDER BY arxiv_id ASC, serial ASC LIMIT 100

Sample result:
N.N. Bogolubov, {\it Lectures on Quantum Statistics} (Gordon and
Breach, New York, 1967), Vol. 1.


Stiching multiple tex file, folder, nested,  --> put things together -->

then you have different styles -->
\bibitem{...} blocks (manual or .bbl)
.bib files (BibTeX)
biblatex-style declarations. (Parsing)


--> then let's say we parsed (regex, latexwaler etc) -->

how to recognize each entity is another problem...  (NER)


This can be a year long project lol, it's a whole engine --> I don't think we have the resource to build this from ground up + 100% accuracy is not feasible

only assuming that we have the right tex file


conclusion: maybe LaTeX is strongly necessary for scientific formula, but for citations, not really

Every step we do some kind of processing, we will in evitablly lose some data...


TODO: compare Grobid with LLM with semantic scholar, with the actual paper --> then we also need to deal with ambiguity somehow...


Take data out, opt out, if someone said no I don't want my data to be in the model...




In [1]:
!git clone https://github.com/Luke-Guan-FirstPrinciples/Latex-papers.git

Cloning into 'Latex-papers'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 123.78 KiB | 4.27 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
!pip install bibtexparser pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=2711e10a575037c0d3ac95b7e2a38a032c6304bc9cb6f170f9d98528db5fcb06
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=b60776a4a88c87355e2fd1c1a4dd810af406831f5402cab2c131ffca7eb6ad77
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built bibtexparser pylatexenc


In [3]:
import re
import os
import bibtexparser


def detect_bib_system(tex_content):
    if r'\bibliography{' in tex_content:
        return 'bibtex'
    elif r'\addbibresource{' in tex_content:
        return 'biblatex'
    elif r'\begin{thebibliography}' in tex_content:
        return 'manual'
    return 'unknown'




def parse_bibitem_block(tex_content):
    # Matches \bibitem[optional]{key} followed by any text until next \bibitem or end
    pattern = r'\\bibitem(?:\[[^\]]*\])?\{([^\}]+)\}(.*?)(?=(\\bibitem|\Z))'
    matches = re.findall(pattern, tex_content, re.DOTALL)
    parsed = []

    for key, body, _ in matches:
        # Clean up formatting commands for easier reading (optional)
        clean_body = re.sub(r'\\(newblock|textbf|url|doi)\*?{[^}]*}', '', body)
        clean_body = re.sub(r'\s+', ' ', clean_body).strip()
        parsed.append({
            'key': key.strip(),
            'raw': clean_body.strip()
        })

    return parsed


def parse_bbl_file(bbl_path):
    if not os.path.exists(bbl_path):
        return []
    with open(bbl_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return parse_bibitem_block(content)


def parse_bib_file(bib_path):
    if not os.path.exists(bib_path):
        return []
    with open(bib_path, 'r', encoding='utf-8') as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    return bib_database.entries


def parse_latex_references(tex_path):
    with open(tex_path, 'r', encoding='utf-8') as f:
        tex_content = f.read()

    bib_system = detect_bib_system(tex_content)
    print(f'Detected bibliography system: {bib_system}')

    if bib_system == 'normal':
        return parse_bibitem_block(tex_content)

    elif bib_system == 'bibtex':
        # extract .bbl file name (usually same as tex base name)
        bbl_path = tex_path.replace('.tex', '.bbl')
        return parse_bbl_file(bbl_path)

    elif bib_system == 'biblatex':
        # find .bib file(s) from \addbibresource
        bib_paths = re.findall(r'\\addbibresource\{([^}]+)\}', tex_content)
        all_entries = []
        for bib in bib_paths:
            # Strip .bib if not present
            if not bib.endswith('.bib'):
                bib += '.bib'
            all_entries.extend(parse_bib_file(bib))
        return all_entries

    return []


In [14]:
!pwd

/content


In [5]:


# Example usage
if __name__ == '__main__':
    from pprint import pprint
    # 0704.0238 , 0704.0059 not picked up in the DB
    # contains mistakes
    references = parse_latex_references('/content/Latex-papers/ms.tex')

    # unknown --> actually no reference lol
    #references = parse_latex_references('/content/Latex-papers/exoptf_v4.tex')

    # references = parse_latex_references('/content/Latex-papers/0704.0238.tex')

    # https://arxiv.org/abs/1801.04203 multiple tex file, 0 byte, extraction error
    # references = parse_latex_references('/content/Latex-papers/1801.04203.tex')

    # references = parse_latex_references('/content/Latex-papers/1004.1779.tex')

    # references = parse_latex_references('/content/Latex-papers/0806.0754.tex')

    # references = parse_latex_references('/content/Latex-papers/0707.0958.tex')
    # references = parse_latex_references('/content/Latex-papers/recoghx.tex')
    # references = parse_latex_references('sparsity-certifying.tex')
    pprint(references)


Detected bibliography system: manual
[{'key': 'andersen1991', 'raw': 'Andersen, J. 1991, A\\&AR, 3, 91'},
 {'key': 'bakos2002',
  'raw': "Bakos, G.\\ {\\'A}., L{\\'a}z{\\'a}r, J., Papp, I., S{\\'a}ri, P., "
         '\\& Green, E.\\ M. 2002, \\pasp, 114, 974'},
 {'key': 'bakos2004',
  'raw': "Bakos, G., Noyes, R.\\ W., Kov\\'{a}cs, G., Stanek, K.\\ Z., "
         'Sasselov, D.\\ D., \\& Domsa, I. 2004, \\pasp, 116, 266'},
 {'key': 'baraffe1998',
  'raw': 'Baraffe, I., Chabrier, G., Allard, F., \\& Hauschildt, P.\\ H. 1998, '
         '\\aap, 337, 403'},
 {'key': 'baraffe2002',
  'raw': 'Baraffe, I., Chabrier, G., Allard, F., \\& Hauschildt, P.\\ H. 2002, '
         '\\aap, 382, 563'},
 {'key': 'borucki2001',
  'raw': 'Borucki, W.\\ J., Caldwell, D., Koch, D.\\ G., Webster, L.\\ D., '
         'Jenkins, J.\\ M., Ninkov, Z., \\& Showen, R. 2001, \\pasp, 113, 439'},
 {'key': 'bouchy2005',
  'raw': 'Bouchy, F., Pont, F., Melo, C., Santos, N.\\ C., Mayor, M., Queloz, '
         'D., \\& Udr

In [10]:


# Example usage
if __name__ == '__main__':
    from pprint import pprint
    # 0704.0238 , 0704.0059 not picked up in the DB
    # contains mistakes
    # references = parse_latex_references('/content/Latex-papers/ms.tex')

    # unknown --> actually no reference lol
    #references = parse_latex_references('/content/Latex-papers/exoptf_v4.tex')

    # references = parse_latex_references('/content/Latex-papers/0704.0238.tex')

    # https://arxiv.org/abs/1801.04203 multiple tex file, 0 byte, extraction error
    # references = parse_latex_references('/content/Latex-papers/1801.04203.tex')

    # references = parse_latex_references('/content/Latex-papers/1004.1779.tex')

    # references = parse_latex_references('/content/Latex-papers/0806.0754.tex')

    # references = parse_latex_references('/content/Latex-papers/0707.0958.tex')
    # references = parse_latex_references('/content/Latex-papers/recoghx.tex')
    references = parse_latex_references('/content/Latex-papers/sparsity-certifying.tex')
    pprint(references)


Detected bibliography system: manual
[{'key': 'berg:jordan:2003',
  'raw': "Berg, A.R., Jord\\'{a}n, T.: Algorithms for graph rigidity and scene "
         'analysis. \\newblock In: Proc. 11th European Symposium on Algorithms '
         "(ESA '03), LNCS, vol. 2832, pp. 78--89. (2003)"},
 {'key': 'crapo:rigidity:88',
  'raw': 'Crapo, H.: On the generic rigidity of plane frameworks. \\newblock '
         "Tech. Rep. 1278, Institut de recherche d'informatique et "
         "d'automatique (1988)"},
 {'key': 'Ed65',
  'raw': 'Edmonds, J.: Minimum partition of a matroid into independent sets. '
         '\\newblock J. Res. Nat. Bur. Standards Sect. B , 67--72 (1965)'},
 {'key': 'edmonds:matroidpolyhedra',
  'raw': 'Edmonds, J.: Submodular functions, matroids, and certain polyhedra. '
         '\\newblock In: Combinatorial Optimization---Eureka, You Shrink!, no. '
         '2570 in LNCS, pp. 11--26. Springer (2003)'},
 {'key': 'gabow:jcss-1995',
  'raw': 'Gabow, H.N.: A matroid approach to fi

In [11]:


# Example usage
if __name__ == '__main__':
    from pprint import pprint
    # 0704.0238 , 0704.0059 not picked up in the DB
    # contains mistakes
    # references = parse_latex_references('/content/Latex-papers/ms.tex')

    # unknown --> actually no reference lol
    #references = parse_latex_references('/content/Latex-papers/exoptf_v4.tex')

    # references = parse_latex_references('/content/Latex-papers/0704.0238.tex')

    # https://arxiv.org/abs/1801.04203 multiple tex file, 0 byte, extraction error
    # references = parse_latex_references('/content/Latex-papers/1801.04203.tex')

    # references = parse_latex_references('/content/Latex-papers/1004.1779.tex')

    # references = parse_latex_references('/content/Latex-papers/0806.0754.tex')

    # references = parse_latex_references('/content/Latex-papers/0707.0958.tex')
    references = parse_latex_references('/content/Latex-papers/recoghx.tex')
    #references = parse_latex_references('/content/Latex-papers/sparsity-certifying.tex')
    pprint(references)


Detected bibliography system: manual
[{'key': 'Alexandrov1945',
  'raw': '{A.D. Alexandrov}, {\\em Isoperimetric inequalities for curved '
         'surfaces}, Dokl. Akad. Nauk USSR {\\bf 47} (1945), 235-238.'},
 {'key': "Alexandrov-Strel'tsov1965",
  'raw': "{A. D. Aleksandrov and V. V. Strel'tsov}, {\\em The isoperimetric "
         'problem and estimates of the length of a curve on a surface}, Proc. '
         'Steklov Inst. Math. {\\bf 76} (1965), 67-80; English transi., Proc. '
         'Steklov Inst Math. (1967), 81-99.'},
 {'key': 'Anderson1990',
  'raw': '{M. T. Anderson}, {\\em Convergence and rigidity of manifolds under '
         'Ricci curvature bounds}, Invent. Math. {\\bf 102} (1990), no. 2, '
         '429-445.'},
 {'key': 'Aubry-Grosjean',
  'raw': '{E. Aubry and J.-F. Grosjean}, {\\em Spectrum of hypersurfaces with '
         'small extrinsic radius or large $\\lambda_1$ in Euclidean spaces}, '
         'Journal of Functional Analysis, {\\bf 271} (2016), Issue 5, '
   